In [1]:
%load_ext autoreload
%autoreload 2

import torch 
import os
import glob
from matplotlib import pyplot as plt

import matplotlib.animation as animation
import os
import torch
import numpy as np
import pandas as pd
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')


from distributions import ZeroInflatedDist, QuantizedNormal
from torch_models import MixtureOfTruncNormModel, SpatialWaves
from torch_training import train_epoch
from torch_perturb.torch_pert_topk import PerturbedTopK
import time
from torch_training import train_epoch_largesynth
from torch_models import torch_bpr_uncurried, deterministic_bpr

In [13]:
data_dir='/cluster/home/kheuto01/code/prob_diff_topk'

step_size=0.1
perturbed_noise=0.0001
K=100
threshold=0.5
rows=27
cols=60
data_shape=(rows, cols)
deaths = pd.read_csv(os.path.join(data_dir,'deaths_band.csv'))
pop = pd.read_csv(os.path.join(data_dir, 'pop_band.csv'))
num_score_samples = 50
num_pert_samples = 50

# turn the death column into a time-by-geoid array
deaths_TS = deaths.pivot(index='time', columns='geoid', values='death').values
pop_S = pop['pop'].values

T, S = deaths_TS.shape
# create latitude and longitude arrays corresponding to the row and column index of the geoids when reshaped into data_shape
lat = np.linspace(-rows/2, rows/2, rows)
lon = np.linspace(-cols/2, cols/2, cols)
lat_S, lon_S = np.meshgrid(lon, lat)
lat_S = lat_S.flatten()
lon_S = lon_S.flatten()
# create column of time values
time_T = np.arange(deaths_TS.shape[0])
time_T = torch.tensor(time_T, dtype=torch.float32)
lat_S = torch.tensor(lat_S, dtype=torch.float32)
lon_S = torch.tensor(lon_S, dtype=torch.float32)
pop_S = torch.tensor(pop_S, dtype=torch.float32)
S = pop_S.shape[0]
T = time_T.shape[0]


model  = SpatialWaves(num_waves=1,low=0, high=1000000)

optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

M_score_func =  num_score_samples
M_action = M_score_func
train_y_TS = torch.tensor(deaths_TS)
perturbed_top_K_func = PerturbedTopK(k=K, sigma=perturbed_noise, num_samples=num_pert_samples)

In [14]:
from torch.autograd import profiler
optimizer.zero_grad()
with torch.profiler.profile(profile_memory=True,
    activities=[
        torch.profiler.ProfilerActivity.CPU,
    ]
) as p:
        dist = model(time_T,pop_S, lat_S, lon_S)

        y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
        ratio_rating_TMS = y_sample_TMS/y_sample_TMS.sum(dim=-1, keepdim=True)
        ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
        ratio_rating_TS.requires_grad_(True)


        positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, train_y_TS, K=K, perturbed_top_K_func=perturbed_top_K_func)
        negative_bpr_loss = torch.mean(-positive_bpr_T)

        loss = negative_bpr_loss

        loss.backward()


STAGE:2024-09-10 16:16:47 24685:24685 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


In [12]:
ratio_rating_TS.grad.shape

torch.Size([64, 1620])